In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
from datetime import datetime
import numpy as np
import warnings
from math import sqrt

from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
from dateutil.rrule import rrule, rruleset

from ts_viz import TimeSeriesViz

In [ ]:
# func = lambda x, a, b: a * np.exp(b * x) 
# func_name = 'exp'

func = lambda t, K, x, t0, b: K * np.power(t, x) * np.exp(-t / t0) + b
func_name = 'power'

def show_fit(series, func=False, func_name=False, title=None, pred=7, sma=False, figsize=(10, 6)):
    x = np.arange(len(series))
    index = pd.date_range(series.index.min(), periods=(len(series) + pred), freq='D')
    if func:    
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            popt, pcov = curve_fit(func, x, series.values, maxfev=10000, p0=(1, 1, 5, 0))
        y = func(np.arange(len(index)), *popt)
        fit = pd.Series(y, index=index)
        err = np.sqrt(np.sum((fit[:-pred] - series) ** 2) / len(series))
        print(f'Fit error: {err:.02f}')
        print(f'Predicted volume error: {1 - fit[:-pred].sum() / series.sum():.02%}')

    fig, ax = plt.subplots(figsize=figsize)
    if func:
        ax.plot(index, y, label=f'{func_name} fit', color='tab:red', lw=2)
        ax.axvline(index[np.argmax(fit)], color='tab:red')

    ax.bar(series.index, series, label=series.name)
    
    if sma:
        sma_series = series.rolling(sma, center=True).mean()
        ax.plot(sma_series.index, sma_series, label=f'Moving avg ({sma} days)', color='tab:orange', lw=2)
    
#     locator = mdates.DayLocator(interval=7)
#     locator = mdates.WeekdayLocator(byweekday=mdates.MO)
    locator = mdates.MonthLocator(bymonthday=[1, 15])
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_minor_locator(mdates.DayLocator(interval=1))
    ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
    ax.set_ylim(0, None)
    if title:
        ax.set_title(title)
    plt.legend(loc='upper left')
    if func:
        return (fig, ax), popt, pcov, fit
    else:
        return (fig, ax), None, None, None

In [ ]:
yesterday = True

day = (datetime.now() - dt.timedelta(days=1 if yesterday else 0)).strftime('%Y-%m-%d')
world_url =  'https://www.ecdc.europa.eu/sites/default/files/documents/'\
            f'COVID-19-geographic-disbtribution-worldwide-{day}.xlsx'
print('Downloading:', world_url)
date_parser = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
df_raw = pd.read_excel(world_url, parse_dates=['dateRep'], index_col=0)
print(f'Last day with data: {df_raw.index.max():%Y-%m-%d}')
df_raw = df_raw[['cases', 'deaths', 'countriesAndTerritories']]
df_raw.columns = ['cases', 'deaths', 'country']
df_raw = df_raw.sort_index()
df_raw[df_raw['country'] == 'Hungary'].tail()

In [ ]:
df = df_raw.groupby('country').sum()

In [ ]:
df.to_csv(f'data/world-{day}.csv')

In [ ]:
class FitFunc:
    def __init__(self, func, name, disp_eq, params_name, params_init, maxfev=2000, validator=None):
        self.func = func
        self.name = name
        self.disp_eq = disp_eq
        self.params_name = params_name
        self.params_init = params_init
        self.maxfev = maxfev
        self.validator = validator
    
    def fit_timeseries(self, series, pred=7, rmse_ma_window=1):
        x = np.arange(len(series))
        index = pd.date_range(series.index.min(), periods=(len(series) + pred), freq='D')
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            popt, pcov = curve_fit(self.func, x, series.values, maxfev=self.maxfev, p0=self.params_init)
        if callable(self.validator) and not self.validator(popt):
            return (None, None, None, None, popt)
        y = self.func(np.arange(len(index)), *popt)
        fit = pd.Series(y, index=index)
        rmse_pred = series.rolling(window=rmse_ma_window, center=True, min_periods=1).mean().values
        rmse = sqrt(mean_squared_error(y[x], rmse_pred))
        max_idx = fit.index[np.argmax(fit)]
        max_val = fit[max_idx]
        return (fit, rmse, max_idx, max_val, popt)

power_func = FitFunc(
    func=lambda t, K, x, t0, b: K * np.power(t, x) * np.exp(-t / t0) + b,
    name='power law',
    disp_eq='$n(t)=Kt^{{x}}e^{{-t/t_0}}+b$',
    params_name=['K', 'x', 't_0', 'b'],
    params_init=[1, 1, 5, 0],
    maxfev=10000,
    validator=lambda popt: (popt[0] > 0)
)

class Diag:
    def __init__(self, series, series_name=None, figsize=(16, 10)):
        self.series = series
        self.figsize = figsize
        self.series_name = series_name if series_name else series.name
        self.title_lines = [self.series_name]
    
    def __enter__(self):
        self.fig, self.ax = plt.subplots(figsize=self.figsize)
        self.ax.yaxis.grid(True, which='major')
        locator = mdates.AutoDateLocator()
        self.ax.xaxis.set_major_locator(locator)
        self.ax.xaxis.set_major_formatter(mdates.ConciseDateFormatter(locator))
        self.ax.xaxis.set_minor_locator(mdates.DayLocator())
        return self
    
    def __exit__(self, exc_type, exc_value, exc_traceback):
        self.ax.set_title(self.series_name)
        self.ax.legend()
        self.ax.set_title('\n'.join(self.title_lines))
        plt.show()
    
    def show_series(self):
        self.ax.bar(self.series.index, self.series, align='center', label='data')
    
    def show_ma(self, window=7, center=True, color='tab:orange', lw=3):
        ma = self.series.rolling(window=window, center=center).mean()
        self.ax.plot(ma.index, ma, label=f'{window} days moving average', color=color, lw=lw)
    
    def show_fit(self, fit_func, pred=7, color='tab:red', lw=3):
        (fit, rmse, max_idx, max_val, popt) = fit_func.fit_timeseries(self.series, pred)
        print(f'{fit_func.name} fit:')
        format_num = lambda x: f'{x:.02e}' if x < 0.005 else f'{x:.02f}'
        params_str = ', '.join([f'{k} = {format_num(v)}' for k, v in zip(fit_func.params_name, popt)])
        print(f'    params: {params_str}')
        if fit is not None:
            self.ax.plot(fit.index, fit, label=f'{fit_func.name} fit', color=color, lw=lw)
            self.ax.axvline(max_idx, color=color, lw=lw)
            self.title_lines.append(f'Fit function: {fit_func.name}, {fit_func.disp_eq}')
            print(f'    RMSE = {rmse:.02f}')
            print(f'    peak day: {max_idx:%Y-%m-%d}, value = {max_val:.0f}')
        else:
            print('    Could not find valid fit.')

In [ ]:
get_kwarg = lambda kw, k, d: kw[k] if k in kw else d

class CountryViz:
    def __init__(self, df_raw, fit_func, start=None, end=None, pred=7, ma_window=7):
        self.df_raw = df_raw
        self.fit_func = fit_func
        self.start = start
        self.end = end
        self.pred = pred
        self.ma_window = ma_window
        
    def _get_series(self, country, column, **kwargs):
        series = self.df_raw[(df_raw['country'] == country)].sort_index()[column]
        start = kwargs['start'] if 'start' in kwargs else self.start
        end = kwargs['end'] if 'end' in kwargs else self.end
        if start:
            series = series[series.index >= start]
        if end:
            series = series[series.index <= end]
        return series

    def _show_diag(self, country, column, name, **kwargs):
        pred = get_kwarg(kwargs, 'pred', self.pred)
        show_series = get_kwarg(kwargs, 'show_series', True)
        show_ma = get_kwarg(kwargs, 'show_ma', True)
        show_fit = get_kwarg(kwargs, 'show_fit', True)
        series = self._get_series(country, column, **kwargs)
        series.name = f'{name} in {country}'
        with Diag(series) as diag:
            if show_series:
                diag.show_series()
            if show_ma:
                diag.show_ma(window=self.ma_window)
            if show_fit:
                diag.show_fit(self.fit_func, pred=pred)

    def show_deaths(self, country, **kwargs):
        self._show_diag(country, 'deaths', 'Daily deaths', **kwargs)
    
    def show_cases(self, country, **kwargs):
        self._show_diag(country, 'cases', 'Daily cases', **kwargs)

In [ ]:
viz = CountryViz(df_raw, power_func)

In [ ]:
viz.show_cases('Italy', start='2020-09-01')

In [ ]:
viz.show_deaths('Italy', start='2020-10-01')

In [ ]:
viz.show_cases('France', start='2020-09-01')

In [ ]:
viz.show_deaths('France', start='2020-09-15')

In [ ]:
viz.show_cases('Hungary', start='2020-09-15')

In [ ]:
viz.show_deaths('Hungary', start='2020-10-01')

In [ ]:
viz.show_cases('Belgium', start='2020-09-01')

In [ ]:
viz.show_deaths('Belgium', start='2020-10-01')

In [ ]:
viz.show_cases('Spain', start='2020-09-01')

In [ ]:
viz.show_deaths('Spain', start='2020-09-15')

In [ ]:
viz.show_cases('United_Kingdom', start='2020-09-01')

In [ ]:
viz.show_deaths('United_Kingdom', start='2020-09-15')

In [ ]:
viz.show_cases('United_States_of_America', start='2020-09-01', pred=14)

In [ ]:
viz.show_deaths('United_States_of_America', start='2020-10-01', pred=14)

In [ ]:
viz.show_cases('Sweden', start='2020-10-01')

In [ ]:
viz.show_deaths('Sweden', start='2020-10-15')

In [ ]:
viz.show_cases('Germany', start='2020-09-15')

In [ ]:
viz.show_deaths('Germany', start='2020-10-01')

In [ ]:
viz.show_cases('Iceland', start='2020-09-01')

In [ ]:
viz.show_cases('Singapore', start='2020-07-01')

In [ ]:
viz.show_cases('South_Korea', start='2020-10-15')

In [ ]:
viz.show_cases('Japan', start='2020-09-15', pred=30)

In [ ]:
viz.show_cases('Brazil', start='2020-04-01')

In [ ]:
viz.show_deaths('Brazil', start='2020-04-01')

In [ ]:
viz.show_cases('Russia', start='2020-08-15', pred=60)

In [ ]:
viz.show_deaths('Russia', start='2020-09-15', pred=150)

In [ ]:
viz.show_cases('Romania', start='2020-09-01')

In [ ]:
viz.show_deaths('Romania', start='2020-10-01')

In [ ]:
viz.show_cases('Ukraine', start='2020-08-01')

In [ ]:
viz.show_cases('Iran', start='2020-10-01', pred=30)

In [ ]:
viz.show_deaths('Iran', start='2020-09-01', pred=30)

In [ ]:
viz.show_cases('Czechia', start='2020-09-15')

In [ ]:
viz.show_deaths('Czechia', start='2020-09-15')

In [ ]:
viz.show_cases('South_Africa', start='2020-04-15')

In [ ]:
# fig, ax = CountryViz('Chile', '2020-02-15', df_raw).show_cases()
# ax.set_ylim(None, 8000)

In [ ]:
# CountryViz('Australia', '2020-02-15', df_raw).show_cases()

In [ ]:
# CountryViz('New_Zealand', '2020-02-15', df_raw).show_cases()

In [ ]:
# CountryViz('Canada', '2020-02-15', df_raw).show_cases()

In [ ]:
# india = CountryViz('India', '2020-02-15', df_raw)
# india.show_cases()

In [ ]:
# india.show_deaths()

In [ ]:
# fig, ax = CountryViz('China', '2020-01-01', df_raw).show_cases()
# ax.set_ylim(0, 5000)

In [ ]:
# fig, ax = CountryViz('Belgium', '2020-01-01', df_raw).show_cases()

In [ ]:
sorted(df_raw['country'].unique())

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots(figsize=(16, 10))
country = 'China'
start = '2020-01-15'
end = '2020-03-01'
country_df = df_raw[(df_raw['country'] == country) & (df_raw.index > start) & (df_raw.index < end)].sort_index()
ax.set_title('Total cases of COVID-19 in China')
cases_exp = country_df['cases'].expanding().sum()
ax.plot(cases_exp)

ann_start = '2020-02-12'
ann_stop = '2020-02-13'

ann_center_x = (mdates.datestr2num(ann_start) + mdates.datestr2num(ann_stop)) / 2
ann_center_y = (cases_exp[ann_start] + cases_exp[ann_stop]) / 2
ann_zoom = 1.5
ann_height = (cases_exp[ann_stop] - cases_exp[ann_start]) * ann_zoom
timespan = mdates.datestr2num(ann_stop) - mdates.datestr2num(ann_start)
ann_width = timespan * 2 * ann_zoom
ellipse = Ellipse(xy=(ann_center_x, ann_center_y), width=ann_width, height=ann_height, edgecolor='r', fc='None', lw=2)
ax.add_patch(ellipse)

ax.annotate('China changed\ncase definition',
            xy=(ann_center_x, ann_center_y), xycoords='data',
            xytext=(50, -100), textcoords='offset points',
            size=15,
            arrowprops=dict(arrowstyle="simple",
                            fc="0.6", ec="none",
                            patchB=ellipse,
                            connectionstyle="arc3,rad=0.3"))

plt.show()